In [1]:
# %pip install langchain_community
# %pip install langchain_huggingface
# %pip install sentence-transformers
# %pip uninstall numpy -y
# %pip install "numpy<2.0"
%pip install faiss-cpu


Note: you may need to restart the kernel to use updated packages.


In [2]:
# check_db.py
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# --- 설정 (기존 generate_RAG.py와 동일하게 설정) ---
FAISS_DB_PATH = "/workspace/2025-AI-Challeng-finance/split_RAG_inference/faiss_index_laws"
EMBEDDING_MODEL_NAME = "BAAI/bge-m3"

def inspect_vector_db():
    """
    저장된 FAISS 벡터 DB를 불러와 내용을 확인합니다.
    """
    try:
        # 1. 임베딩 모델 로드 (DB를 불러올 때도 동일한 모델 필요)
        print(f"🚀 임베딩 모델({EMBEDDING_MODEL_NAME})을 로드합니다...")
        embedding_model = HuggingFaceEmbeddings(
            model_name=EMBEDDING_MODEL_NAME,
            model_kwargs={'device': 'cpu'},
            encode_kwargs={'normalize_embeddings': True}
        )
        print("✅ 임베딩 모델 로드 완료.")

        # 2. 로컬에 저장된 FAISS DB 불러오기
        print(f"⏳ '{FAISS_DB_PATH}'에서 벡터 DB를 로드합니다...")
        vector_db = FAISS.load_local(
            FAISS_DB_PATH,
            embedding_model,
            allow_dangerous_deserialization=True # Langchain 최신 버전 호환성
        )
        print("✅ 벡터 DB 로드 완료.")

        # 3. DB 정보 및 내용 확인
        # FAISS 인덱스 내부의 docstore를 통해 원본 문서에 접근 가능
        docstore = vector_db.docstore
        total_docs = len(docstore._dict)
        print("\n" + "="*50)
        print(f"📊 총 {total_docs}개의 문서(chunk)가 DB에 저장되어 있습니다.")
        print("="*50 + "\n")

        # 처음 5개 문서의 내용과 메타데이터 출력
        print("🔍 저장된 문서 샘플 (처음 5개):")
        doc_ids = list(docstore._dict.keys())
        for i, doc_id in enumerate(doc_ids[:5]):
            doc = docstore._dict[doc_id]
            print(f"\n--- 문서 {i+1} ---")
            print(f"  [내용 (page_content)]:\n  '{doc.page_content[:300]}...'") # 내용이 길 수 있으므로 300자만 출력
            print(f"  [메타데이터 (metadata)]: {doc.metadata}")

    except Exception as e:
        print(f"\n❌ 오류가 발생했습니다: {e}")
        print(f"'{FAISS_DB_PATH}' 경로에 DB 파일이 존재하는지 확인해주세요.")


if __name__ == "__main__":
    inspect_vector_db()

🚀 임베딩 모델(BAAI/bge-m3)을 로드합니다...


/opt/miniforge3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ 임베딩 모델 로드 완료.
⏳ '/workspace/2025-AI-Challeng-finance/split_RAG_inference/faiss_index_laws'에서 벡터 DB를 로드합니다...
✅ 벡터 DB 로드 완료.

📊 총 88개의 문서(chunk)가 DB에 저장되어 있습니다.

🔍 저장된 문서 샘플 (처음 5개):

--- 문서 1 ---
  [내용 (page_content)]:
  '법제처                                                            1                                                       국가법령정보센터
신용정보의 이용 및 보호에 관한 법률
 
신용정보의 이용 및 보호에 관한 법률 ( 약칭: 신용정보법 )
[시행 2024. 8. 14.] [법률 제20304호, 2024. 2. 13., 일부개정]
금융위원회 (금융데이터정책과) 02-2100-2624
       제1장 총칙
 
제1조(목적) 이 법은 신용정보...'
  [메타데이터 (metadata)]: {'source': '신용정보의 이용 및 보호에 관한 법률(법률)(제20304호)(20240814).pdf'}

--- 문서 2 ---
  [내용 (page_content)]:
  '2) 법령에 따라 특정 개인을 고유하게 식별할 수 있도록 부여된 정보로서 대통령령으로 정하는 정보(이하 “개
인식별번호”라 한다)
3) 개인의 신체 일부의 특징을 컴퓨터 등 정보처리장치에서 처리할 수 있도록 변환한 문자, 번호, 기호 또는 그
밖에 이와 유사한 정보로서 특정 개인을 식별할 수 있는 정보
4) 1)부터 3)까지와 유사한 정보로서 대통령령으로 정하는 정보
나. 기업(사업을 경영하는 개인 및 법인과 이들의 단체를 말한다. 이하 같다) 및 법인의 정보로서 다음 각각의 정
보
1) 상호 및 명칭
2) 본점ㆍ영업소 및 주된 사...'
  [메타데이터 (me

In [3]:
%pip install bitsandbytes
%pip install accelerate
!pip install accelerate



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from tqdm import tqdm
import os

# --- 1. 설정 (Configuration) ---

# == 원본 DB 및 모델 설정 ==
ORIGINAL_DB_PATH = "./faiss_index_laws"
EMBEDDING_MODEL_NAME = "BAAI/bge-m3"
BASE_MODEL_ID = "K-intelligence/Midm-2.0-Base-Instruct"

# == 새로 생성할 요약 DB 경로 설정 ==
SUMMARY_DB_PATH = "./faiss_index_summary"


# --- 2. 요약을 위한 LLM 로드 함수 ---
def load_summarization_llm():
    """
    청크 요약을 위한 LLM을 로드하고 파이프라인을 생성합니다.
    """
    print("⏳ 요약용 LLM을 로딩합니다... (메모리에 따라 시간이 소요될 수 있습니다)")

    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_ID,
        quantization_config=quantization_config,
        device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    summarizer_pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device_map="auto"
    )
    print("✅ 요약용 LLM 로딩 완료.")
    return summarizer_pipe

def make_summary_prompt(text: str) -> str:
    """
    LLM에게 전달할 요약 프롬프트를 생성합니다.
    """
    prompt = f"""### 지시:
당신은 문서 정리 AI 어시스턴트입니다. 주어진 문서를 깔끔한 줄글로 변경해주세요. 
최대한 모든 내용을 포함하며 이해하기 쉽게 줄글 형태로 데이터를 정리해주세요.

### 문서 내용:
{text}

### 요약:
"""
    return prompt


# --- 3. 메인 실행 로직 ---
def create_summary_vector_db():
    """
    원본 DB를 로드하여 각 문서를 요약하고, 새로운 요약 기반 DB를 생성합니다.
    """
    # 1. 요약 LLM 준비
    summarizer = load_summarization_llm()

    # 2. 임베딩 모델 및 원본 DB 로드
    print(f"🚀 임베딩 모델({EMBEDDING_MODEL_NAME})을 로드합니다...")
    embedding_model = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL_NAME,
        model_kwargs={'device': 'cpu'}, # LLM이 GPU를 사용하므로 임베딩은 CPU로
        encode_kwargs={'normalize_embeddings': True}
    )
    print("✅ 임베딩 모델 로드 완료.")

    print(f"⏳ 원본 벡터 DB를 '{ORIGINAL_DB_PATH}'에서 로드합니다...")
    if not os.path.exists(ORIGINAL_DB_PATH):
        print(f"❌ 오류: 원본 DB 경로 '{ORIGINAL_DB_PATH}'를 찾을 수 없습니다.")
        return
        
    original_db = FAISS.load_local(
        ORIGINAL_DB_PATH,
        embedding_model,
        allow_dangerous_deserialization=True
    )
    print("✅ 원본 벡터 DB 로드 완료.")

    # 3. 문서 요약 및 새로운 Document 객체 생성
    docstore = original_db.docstore
    original_docs = list(docstore._dict.values())
    new_summary_docs = []

    print(f"\n🔄 총 {len(original_docs)}개의 문서를 요약합니다...")
    for doc in tqdm(original_docs, desc="📄 문서 요약 진행 중"):
        original_text = doc.page_content
        prompt = make_summary_prompt(original_text)

        # LLM으로 요약 생성
        output = summarizer(
            prompt,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.3,
            return_full_text=False,
            eos_token_id=summarizer.tokenizer.eos_token_id
        )
        summary_text = output[0]['generated_text'].strip()

        # 새로운 Document 객체 생성
        # page_content에는 요약본, metadata에는 원본 텍스트 저장
        new_doc = Document(
            page_content=summary_text,
            metadata={
                "original_text": original_text,
                "source": doc.metadata.get("source", "N/A") # 기존 메타데이터 유지
            }
        )
        new_summary_docs.append(new_doc)

    # 4. 새로운 요약 기반 FAISS DB 생성 및 저장
    if not new_summary_docs:
        print("⚠️ 요약된 문서가 없어 DB를 생성할 수 없습니다.")
        return

    print("\n🧠 새로운 요약 기반 FAISS 벡터 DB를 생성합니다...")
    summary_db = FAISS.from_documents(new_summary_docs, embedding_model)
    summary_db.save_local(SUMMARY_DB_PATH)
    print(f"✅ 새로운 요약 DB 구축 및 저장 완료: '{SUMMARY_DB_PATH}'")


if __name__ == "__main__":
    create_summary_vector_db()



⏳ 요약용 LLM을 로딩합니다... (메모리에 따라 시간이 소요될 수 있습니다)


Loading checkpoint shards: 100%|██████████| 5/5 [00:18<00:00,  3.75s/it]
Device set to use cuda:0


✅ 요약용 LLM 로딩 완료.
🚀 임베딩 모델(BAAI/bge-m3)을 로드합니다...
✅ 임베딩 모델 로드 완료.
⏳ 원본 벡터 DB를 './faiss_index_laws'에서 로드합니다...
✅ 원본 벡터 DB 로드 완료.

🔄 총 88개의 문서를 요약합니다...


📄 문서 요약 진행 중:  55%|█████▍    | 48/88 [14:24<14:17, 21.44s/it]

In [12]:
# check_summary_db.py

from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# --- 설정 (generate_summary_db.py와 동일하게 설정) ---
# ⭐️ 확인할 DB 경로를 새로운 요약 DB 경로로 수정합니다.
FAISS_DB_PATH = "./faiss_index_summary"
EMBEDDING_MODEL_NAME = "BAAI/bge-m3"

def inspect_summary_db():
    """
    저장된 요약 기반 FAISS 벡터 DB를 불러와 그 구조와 내용을 확인합니다.
    """
    try:
        # 1. 임베딩 모델 로드
        print(f"🚀 임베딩 모델({EMBEDDING_MODEL_NAME})을 로드합니다...")
        embedding_model = HuggingFaceEmbeddings(
            model_name=EMBEDDING_MODEL_NAME,
            model_kwargs={'device': 'cpu'},
            encode_kwargs={'normalize_embeddings': True}
        )
        print("✅ 임베딩 모델 로드 완료.")

        # 2. 로컬에 저장된 요약 DB 불러오기
        print(f"⏳ '{FAISS_DB_PATH}'에서 요약 DB를 로드합니다...")
        vector_db = FAISS.load_local(
            FAISS_DB_PATH,
            embedding_model,
            allow_dangerous_deserialization=True
        )
        print("✅ 요약 DB 로드 완료.")

        # 3. DB 정보 및 내용 확인
        docstore = vector_db.docstore
        total_docs = len(docstore._dict)
        print("\n" + "="*60)
        print(f"📊 총 {total_docs}개의 요약 문서가 DB에 저장되어 있습니다.")
        print("="*60 + "\n")

        # 처음 3개 문서의 구조를 상세히 출력
        print("🔍 저장된 문서 샘플 (처음 3개):")
        doc_ids = list(docstore._dict.keys())
        for i, doc_id in enumerate(doc_ids[:3]):
            doc = docstore._dict[doc_id]
            
            # ⭐️ 요약본과 원본을 명확히 구분하여 출력
            summary_text = doc.page_content
            original_text = doc.metadata.get('original_text', '원본 텍스트 없음')
            source_file = doc.metadata.get('source', '출처 없음')

            print(f"\n--- 문서 {i+1} (출처: {source_file}) ---")
            print(f"  [요약본 (page_content - 검색 대상)]:\n  '{summary_text}'")
            print("\n" + "-"*20 + "\n")
            print(f"  [원본 청크 (metadata['original_text'] - 답변 근거)]:\n  '{original_text[:300]}...'") # 원본은 길어서 300자만 표시
            print("-" * 60)

    except Exception as e:
        print(f"\n❌ 오류가 발생했습니다: {e}")
        print(f"'{FAISS_DB_PATH}' 경로에 DB 파일이 존재하는지 확인해주세요.")


if __name__ == "__main__":
    inspect_summary_db()


🚀 임베딩 모델(BAAI/bge-m3)을 로드합니다...


✅ 임베딩 모델 로드 완료.
⏳ './faiss_index_summary'에서 요약 DB를 로드합니다...
✅ 요약 DB 로드 완료.

📊 총 88개의 요약 문서가 DB에 저장되어 있습니다.

🔍 저장된 문서 샘플 (처음 3개):

--- 문서 1 (출처: 신용정보의 이용 및 보호에 관한 법률(법률)(제20304호)(20240814).pdf) ---
  [요약본 (page_content - 검색 대상)]:
  '이 법률은 신용정보 관련 산업의 건전한 육성과 신용정보의 효율적 이용 및 체계적 관리를 통해 건전한 신용질서 확립과 국민경제 발전에 이바지하는 것을 목적으로 하며, 신용정보의 정의와 범위를 명확히 규정하고 있습니다.'

--------------------

  [원본 청크 (metadata['original_text'] - 답변 근거)]:
  '법제처                                                            1                                                       국가법령정보센터
신용정보의 이용 및 보호에 관한 법률
 
신용정보의 이용 및 보호에 관한 법률 ( 약칭: 신용정보법 )
[시행 2024. 8. 14.] [법률 제20304호, 2024. 2. 13., 일부개정]
금융위원회 (금융데이터정책과) 02-2100-2624
       제1장 총칙
 
제1조(목적) 이 법은 신용정보...'
------------------------------------------------------------

--- 문서 2 (출처: 신용정보의 이용 및 보호에 관한 법률(법률)(제20304호)(20240814).pdf) ---
  [요약본 (page_content - 검색 대상)]:
  '이 문서는 신용정보의 이용 및 보호에 관한 법률에서 정의하는 '개인식별번호'와 